In [ ]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 1- data prepration

In [ ]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [ ]:
n_samples, n_features = X.shape

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2 , random_state= 1234)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [ ]:
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

## 2- Model

In [ ]:
class LogisticRegression(nn.Module):
  def __init__(self, n_input_features):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(n_input_features, 1)
  
  def forward(self,x):
    y_predicted = torch.sigmoid(self.linear(x))
    return y_predicted


model = LogisticRegression(n_features)

## 3- Loss and optimizer

In [ ]:
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

## 4- Learning loop

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
  # forward pass and loss
  y_predicted = model(X_train)
  loss = criterion(y_predicted, y_train)

  # backward pass
  loss.backward()

  # updates
  optimizer.step()

  # zero gradiants
  optimizer.zero_grad()

  if (epoch+1) % 10 == 0:
    print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')


epoch: 10, loss = 0.1269
epoch: 20, loss = 0.1257
epoch: 30, loss = 0.1244
epoch: 40, loss = 0.1233
epoch: 50, loss = 0.1221
epoch: 60, loss = 0.1210
epoch: 70, loss = 0.1199
epoch: 80, loss = 0.1189
epoch: 90, loss = 0.1179
epoch: 100, loss = 0.1169


In [ ]:
with torch.no_grad():
  y_predicted = model(X_test)
  y_predicted_cls = y_predicted.round()
  acc = y_predicted_cls.eq(y_test).sum() / y_test.shape[0]

print(acc)

tensor(0.9211)
